In [22]:
import base64
import requests
import datetime
from urllib.parse import urlencode

In [23]:
client_id = 'e738e985fed449e8846df5ee72139b87'
client_secret = '74b5a5f1c5b54fd6bde889ed6615d708'

In [30]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_token_headers(self):
        client_creds = f"{self.client_id}:{self.client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return {"Authorization": f"Basic {client_creds_b64.decode()}"}
    
    def get_token_data(self):
        return {"grant_type": "client_credentials"}
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            return False
        token_response_data = r.json()
        now = datetime.datetime.now()
        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        auth_done = self.perform_auth()
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, _id, resource_type="albums"):
        endpoint = f"https://api.spotify.com/v1/{resource_type}/{_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type='artist'):
        headers = self.get_resource_header()
        url = "https://api.spotify.com/v1/search"
        data = urlencode({
            "q": "Blank Space", 
            "type": "track"
        })

        lookup_url = f"{url}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

In [31]:
client = SpotifyAPI(client_id, client_secret)

In [32]:
client.search("Time", search_type="Track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Blank+Space&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
       'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
       'id': '06HL4z0CvFAxyc27GXpf02',
       'name': 'Taylor Swift',
       'type': 'artist',
       'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
     'available_markets': ['CA', 'US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/2QJmrSgbdM35R67eoGQo4j'},
     'href': 'https://api.spotify.com/v1/albums/2QJmrSgbdM35R67eoGQo4j',
     'id': '2QJmrSgbdM35R67eoGQo4j',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273c79b600289a80aaef74d155d',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02c79b600289a80aaef74d155d',
       'width': 300},
      {'heig

In [34]:
client.get_artist("06HL4z0CvFAxyc27GXpf02")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
 'followers': {'href': None, 'total': 33073540},
 'genres': ['dance pop', 'pop', 'post-teen pop'],
 'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
 'id': '06HL4z0CvFAxyc27GXpf02',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/a37efbc7fd3f5f5df81b48ce9c6de53820b239c1',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/dc6f0ac18ad5db302e3c8a28b655b6ca7244ef08',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/b1627ca86690d2c5b6cc2fb1039a31014e96a22b',
   'width': 160}],
 'name': 'Taylor Swift',
 'popularity': 96,
 'type': 'artist',
 'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}

In [37]:
client.get_album("2QJmrSgbdM35R67eoGQo4j")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
   'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
   'id': '06HL4z0CvFAxyc27GXpf02',
   'name': 'Taylor Swift',
   'type': 'artist',
   'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
 'available_markets': ['CA', 'US'],
 'copyrights': [{'text': '© 2014 Big Machine Records, LLC.', 'type': 'C'},
  {'text': '℗ 2014 Big Machine Records, LLC.', 'type': 'P'}],
 'external_ids': {'upc': '00843930013555'},
 'external_urls': {'spotify': 'https://open.spotify.com/album/2QJmrSgbdM35R67eoGQo4j'},
 'genres': [],
 'href': 'https://api.spotify.com/v1/albums/2QJmrSgbdM35R67eoGQo4j',
 'id': '2QJmrSgbdM35R67eoGQo4j',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b273c79b600289a80aaef74d155d',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e02c79b600289a80aaef74d155d',
   'width': 300},
  {'he